# Introducing a Two-Armed Bandit Problem

Imagine we are playing a game with two coins: one is a fair coin and the other is an unfair coin that will land on 'heads' 70% of the time. Every round, we get to select one coin to flip, and if that coin lands as a 'heads', we get a payoff of $1. If it lands 'tails' we get nothing. 

We want to use this example to understand 

These are bernoulli trials so the count of successes is an underlying binomial distribution. The Beta distribution is the conjugate prior for the Beta and Bernoulli distribution.



In [1]:
import numpy as np
import pandas as pd
from scipy.stats import binom
import plotly.express as px

To do:
1. Create a function that initializes the game with two coins: one is unfair, the other is fair
2. Create a function that will flip a chosen coin and return the result/reward
3. Record the updated distribution based on the information given.

In [170]:
def init_game() -> np.array:
    """Initialize the game with two arms, each with their unique probabilities 'p' of success"""
    if np.random.choice([0,1]) == 0:
        p = np.array([0.5, 0.7])
    else:
        p = np.array([0.7, 0.5])
    return p

In [175]:
def pull_arm(selection: float) -> int:
    """Flip the selected coin with probability p, and return the reward if the coin flip results in a head"""
    if np.random.uniform() <= selection:
        reward = 1
    else:
        reward = 0
    return reward

In [187]:
def run_game(n=10):
    p = init_game()
    reward = 0
    for i in range(n):
        reward += pull_arm(p[1])
    print("Total Reward: ", reward)
    print("Actual probability: ", p[1])
    return reward


In [227]:
n = 5
k = run_game(n)

Total Reward:  2
Actual probability:  0.5


So now that we've been able to create our function to initialize a game and simulate flipping a coin several times, we want to be able to incorporate new information into our decisionmaking. That is, based on the results of the coin flips, how can we use that new information to determine which coin is the 'winner'?

We can use the method of Bayesian updating to put that new knowledge to use. We can use the idea that the Posterior is the Prior probability times the Likelihood of the model given the data.


Recall Bayes Theorem:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

The hard part is translating our problem into a bayesian inference problem. 

We can translate the above number of 'wins' as a Binomial Distribution and compute our posterior probabilities of each p.

What's the probability of seeing 6 successes in 9 trials, with a probability p?

In [224]:
binom.pmf(2, 3, 0.5)

0.375

In [241]:
n = 100
k = run_game(n)

p_grid = np.linspace(0, 1, 1000)
prob_p = np.ones(1000)  # Prior probabilities assigned to each outcome. This is a uniform prior
prob_data = binom.pmf(k=k, n=n, p=p_grid) # Likelihood 
# The binom.pmf function returns the probability of k successes over n trials when the probability of success is p
# We want to plot the likelihood of each of these models across all values of p from 0 to 1
posterior = prob_data * prob_p
posterior = posterior/sum(posterior)
px.line(x=p_grid, y=posterior, title=f"Posterior Probability of a Binomial Distribution with probability p given {k} successes and {n} trials")

Total Reward:  72
Actual probability:  0.7


Reference: [Statistical Rethinking Slides, Winter 2019](https://speakerdeck.com/rmcelreath/l02-statistical-rethinking-winter-2019?slide=22)